In [2]:
import sys
sys.path.append("/scratch/group/csce435-f24/python-3.10.4/lib/python3.10/site-packages")
sys.path.append("/scratch/group/csce435-f24/thicket")
from glob import glob

import matplotlib.pyplot as plt
import pandas as pd

import thicket as th

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
import math

Read all files

In [3]:
#1_trial is a name of a folder containing the cali files, you may create a folder with a different name and replace the folder name here
tk = th.Thicket.from_caliperreader(glob("cali_files/*.cali"))

(2/2) Creating Thicket: 100%|██████████| 215/215 [00:06<00:00, 30.75it/s]


View Calltree

In [4]:
print(tk.tree(metric_column="Avg time/rank"))

  _____ _     _      _        _   
 |_   _| |__ (_) ___| | _____| |_ 
   | | | '_ \| |/ __| |/ / _ \ __|
   | | | | | | | (__|   <  __/ |_ 
   |_| |_| |_|_|\___|_|\_\___|\__|  v2024.1.0

1.990 main
├─ 0.076 MPI_Comm_dup
├─ 0.000 MPI_Finalize
├─ 0.000 MPI_Finalized
├─ 0.000 MPI_Init
├─ 0.000 MPI_Initialized
├─ 0.001 comm
│  └─ 0.001 comm_large
│     ├─ 0.001 MPI_Recv
│     └─ 0.000 MPI_Send
├─ 0.001 comp
│  └─ 0.001 comp_large
├─ 0.002 correctness_check
└─ 0.000 data_init_runtime

Legend (Metric: Avg time/rank Min: 0.00 Max: 1.99 indices: {'profile': 20513600})
█ 1.79 - 1.99
█ 1.39 - 1.79
█ 1.00 - 1.39
█ 0.60 - 1.00
█ 0.20 - 0.60
█ 0.00 - 0.20

name User code    ◀  Only in left graph    ▶  Only in right graph



Group Performance data by `matrix_size` in the Thicket metadata table.

In [5]:
tk.metadata_column_to_perfdata("num_procs")
tk.metadata_column_to_perfdata("input_size")
tk.metadata_column_to_perfdata("input_type")

tk.dataframe = tk.dataframe.reset_index().set_index(["node", "num_procs", "input_size", 'input_type']).sort_index()

tk.dataframe.head(10)

profile  \
node                                 num_procs input_size input_type                     
{'name': 'main', 'type': 'function'} 2         65536      1_perc_perturbed   405361192   
                                                          Random            1717438145   
                                                          ReverseSorted      655427337   
                                                          Sorted             906445751   
                                               262144     1_perc_perturbed  3739320156   
                                                          Random            1788864284   
                                                          ReverseSorted     1907461730   
                                                          Sorted             225778685   
                                               1048576    1_perc_perturbed  2360119226   
                                                          Random            3299018427   

                                                                            nid  \
node                                 num_procs input_size input_type              
{'name': 'main', 'type': 'function'} 2         65536      1_perc_perturbed  1.0   
                                                          Random            1.0   
                                                          ReverseSorted     1.0   
                                                          Sorted            1.0   
                                               262144     1_perc_perturbed  1.0   
                                                          Random            1.0   
                                                          ReverseSorted     1.0   
                                                          Sorted            1.0   
                                               1048576    1_perc_perturbed  1.0   
                                                          Random            1.0   

                                                                             spot.channel  \
node                                 num_procs input_size input_type                        
{'name': 'main', 'type': 'function'} 2         65536      1_perc_perturbed  regionprofile   
                                                          Random            regionprofile   
                                                          ReverseSorted     regionprofile   
                                                          Sorted            regionprofile   
                                               262144     1_perc_perturbed  regionprofile   
                                                          Random            regionprofile   
                                                          ReverseSorted     regionprofile   
                                                          Sorted            regionprofile   
                                               1048576    1_perc_perturbed  regionprofile   
                                                          Random            regionprofile   

                                                                            Min time/rank  \
node                                 num_procs input_size input_type                        
{'name': 'main', 'type': 'function'} 2         65536      1_perc_perturbed       0.570379   
                                                          Random                 0.563332   
                                                          ReverseSorted          0.566167   
                                                          Sorted                 0.555436   
                                               262144     1_perc_perturbed       0.703497   
                                                          Random                 0.743423   
                                                          ReverseSorted          0.774058   
                                               

In [6]:
df = tk.dataframe

In [14]:
# Define common variables
processes = df.index.get_level_values(1).unique()
funcs = df.index.get_level_values(0).unique()
input_sizes = {x : int(math.log2(x)) for x in df.index.get_level_values(2).unique()}
types = df.index.get_level_values(3).unique()

In [15]:
# Change font size for all plots
plt.rcParams.update({"font.size": 20})

In [20]:
for size in input_sizes.keys():
    for func in funcs:
        conditions = (df.index.get_level_values(0) == func) & \
                        (df.index.get_level_values(2) == size)
        arr_df = df.loc[conditions]
        name = func.frame['name']
    

        if name in ['comp_large', 'main', 'comm']:

            file_path = 'pres_figs/2^{}/{}.png'.format(input_sizes[size], name)
            file_path_speedup = 'pres_figs/2^{}/{}_speedup.png'.format(input_sizes[size], name)
            file_path_weak = 'pres_figs/2^{}/{}_weak_scaling.png'.format(input_sizes[size], name)
            
            plt.figure(figsize=(10, 10))
            for arr_type in types:
                type_df = arr_df.loc[arr_df.index.get_level_values(3) == arr_type]
                plt.plot(processes, type_df['Avg time/rank'], label=arr_type)
                
            plt.legend()

            plt.title('{} Strong Scaling (2^{})'.format(name, input_sizes[size]))
            plt.xlabel('Num Processes')
            plt.ylabel('Time (s)')
            plt.savefig(file_path)
            plt.clf()
            plt.close()
            
            plt.figure(figsize=(10, 10))
            for arr_type in types:
                type_df = arr_df.loc[arr_df.index.get_level_values(3) == arr_type]
                avg_base = type_df['Avg time/rank'].iloc[0] * 2
                speedup_avg = avg_base / type_df['Avg time/rank']

                plt.plot(processes, speedup_avg, label=arr_type)

            plt.legend()

            plt.title('{} Speedup (2^{})'.format(name, input_sizes[size]))
            plt.xlabel('Num Processes')
            plt.ylabel('Speedup')
            plt.tight_layout()
            plt.savefig(file_path_speedup)
            plt.clf()
            plt.close()

In [12]:
processes = [2, 4, 8, 16, 32, 64]
input_sizes = [16, 18, 20, 22, 24, 26]

for func in funcs:
    func_df = df.loc[df.index.get_level_values(0) == func]
    name = func.frame['name']

    
    if name in ['main', 'comp_large', 'comm']:
        file_path = 'pres_figs/weak_scaling/{}.png'.format(name)
        
        for arr_type in types:
            type_df = func_df.loc[func_df.index.get_level_values(3) == arr_type]
            times = []
            for input_size, proc in zip(input_sizes, processes):
                conditions = (type_df.index.get_level_values(2) == 2 ** input_size) & \
                                (type_df.index.get_level_values(1) == proc)
                times.append(type_df.loc[conditions]['Avg time/rank'].iloc[0])
            
            plt.plot(processes, times, label=arr_type)
            
            if arr_type == 'Random':
                for j, (x, y) in enumerate(zip(processes, times)):
                    plt.annotate(f'2^{input_sizes[j]}', (x, y),
                                 textcoords="offset points", xytext=(5, 5), ha='center')
        
        plt.xlabel('Processes')
        plt.ylabel('Time (s)')
        plt.title('{} Weak Scaling'.format(name))
        plt.legend()
        plt.savefig(file_path)
        plt.clf()

<Figure size 432x288 with 0 Axes>